In [1]:
import requests
import time
import nest_asyncio
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from  matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, date, timedelta
from typing import Dict, Callable, List
from IPython.display import clear_output, display
from pandas.tseries.offsets import BDay, BMonthBegin
from pandas.io.formats.style import Styler

In [67]:
def plot_current_yields(current_yields_dict: Dict[str, str | int]):
    maturities = list(current_yields_dict.keys())
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=[1, 2, 3, 5, 7, 10, 20, 30],
            y=[
                float(str(x["last"]).strip("%")) / 100
                for x in current_yields_dict.values()
            ],
            mode="lines+markers",
            name="Yield",
        )
    )

    timestamps = [current_yields_dict[mat]["timedate"] for mat in maturities]
    timestamp = max(
        datetime.strptime(time.split(" ")[0] + " " + time.split(" ")[1], "%I:%M %p")
        for time in timestamps
    ).strftime("%I:%M %p %Z")

    fig.update_layout(
        title=f"Current Yield Curve as of {timestamp}",
        xaxis_title="Maturity (Months)",
        yaxis_title="Yield (%)",
        xaxis=dict(tickvals=[1, 2, 3, 5, 7, 10, 20, 30], ticktext=maturities),
        showlegend=False,
    )

    fig.show()


def get_business_days():
    today = pd.Timestamp.now().normalize()
    last_business_day = today - BDay(1)
    a_week_ago = today - pd.DateOffset(weeks=1)
    business_day_week_ago = pd.bdate_range(end=a_week_ago, periods=1)[0]
    a_month_ago = today - pd.DateOffset(months=1)
    business_day_month_ago = pd.bdate_range(end=a_month_ago, periods=1)[0]
    a_year_ago = today - pd.DateOffset(years=1)
    business_day_year_ago = pd.bdate_range(end=a_year_ago, periods=1)[0]

    return (
        last_business_day,
        business_day_week_ago,
        business_day_month_ago,
        business_day_year_ago,
    )


def parse_to_float(value):
    if str(value).endswith("%"):
        return float(value.strip("%")) / 100
    else:
        return float(value)


def plot_current_yields_and_historicals(
    current_yields_dict: Dict[str, str | int],
    df_historical_par_yields: pd.DataFrame,
    df_recent_spots: pd.DataFrame = None,
    plot_yesterday=False,
    plot_week=False,
    plot_month=False,
    plot_year=False,
    plot_five_min=False,
    plot_ten_min=False,
    plot_fifteen_min=False,
    plot_thirty_min=False,
    plot_sixty_min=False,
    plot_ninety_min=False,
    plot_120_min=False,
    custom_date: List[date] = None,
):
    adjusted_months = [1, 2, 3, 5, 7, 10, 20, 30]
    maturities = list(current_yields_dict.keys())
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=adjusted_months,
            y=[
                float(str(x["last"]).strip("%")) / 100
                for x in current_yields_dict.values()
            ],
            mode="lines+markers",
            name="Current",
        )
    )

    yesterday, week, month, year = get_business_days()

    def plot_historicals(date):
        formatted_date = date.strftime("%Y-%m-%d")
        yields = df_historical_par_yields.loc[
            df_historical_par_yields["Date"] == formatted_date
        ]
        yields = yields.values.tolist()[0]
        del yields[:6]
        yields = [y / 100 for y in yields]
        fig.add_trace(
            go.Scatter(
                x=adjusted_months,
                y=yields,
                mode="lines+markers",
                name=f"{formatted_date}",
            )
        )

    def plot_recent_spots(min_key: str):
        yields = df_recent_spots.loc[df_recent_spots["Date"] == min_key]
        yields = yields.values.tolist()[0]
        del yields[:1]
        yields = [parse_to_float(y) / 100 for y in yields]
        fig.add_trace(
            go.Scatter(
                x=adjusted_months,
                y=yields,
                mode="lines+markers",
                name=f"{min_key}",
            )
        )

    if plot_yesterday:
        plot_historicals(yesterday)
    if plot_week:
        plot_historicals(week)
    if plot_month:
        plot_historicals(month)
    if plot_year:
        plot_historicals(year)
    if custom_date:
        [plot_historicals(date) for date in custom_date]

    if plot_five_min:
        plot_recent_spots("5.0m"),
    if plot_ten_min:
        plot_recent_spots("10.0m"),
    if plot_fifteen_min:
        plot_recent_spots("15.0m"),
    if plot_thirty_min:
        plot_recent_spots("30.0m"),
    if plot_sixty_min:
        plot_recent_spots("60.0m"),
    if plot_ninety_min:
        plot_recent_spots("90.0m")
    if plot_120_min:
        plot_recent_spots("120.0m")

    [plot_recent_spots(min_key) for min_key in custom_date] if custom_date else None

    timestamps = [current_yields_dict[mat]["timedate"] for mat in maturities]
    timestamp = max(
        datetime.strptime(time.split(" ")[0] + " " + time.split(" ")[1], "%I:%M %p")
        for time in timestamps
    ).strftime("%I:%M %p %Z")

    fig.update_layout(
        title=f"Current Yield Curve as of {timestamp}",
        xaxis_title="Maturity (Months)",
        yaxis_title="Yield (%)",
        xaxis=dict(tickvals=[1, 2, 3, 5, 7, 10, 20, 30], ticktext=maturities),
        showlegend=True,
    )

    fig.show()

In [3]:
def yields_corr(
    current_yields_dict: Dict[str, str | int],
    df_historical_par_yields: pd.DataFrame = None,
    df_recent_spots: pd.DataFrame = None,
    show_recents_days=False,
    show_recent_spots_mins=False,
    num_of_months: int = None,
):
    def find_historical_yields(date):
        try:
            formatted_date = date.strftime("%Y-%m-%d")
            yields = df_historical_par_yields.loc[
                df_historical_par_yields["Date"] == formatted_date
            ]
            yields = yields.values.tolist()[0]
            del yields[:6]
            yields = [y / 100 for y in yields]
            return {date: yields}
        except Exception as e:
            print(f"Date: {date} probably not found: ", e, " trying next day")
            next_day = date + timedelta(days=1)
            return find_historical_yields(next_day)


    maturities = list(current_yields_dict.keys())
    timestamps = [current_yields_dict[mat]["timedate"] for mat in maturities]
    timestamp = max(
        datetime.strptime(time.split(" ")[0] + " " + time.split(" ")[1], "%I:%M %p")
        for time in timestamps
    ).strftime("%H:%M:%S")
    today = datetime.now().strftime("%Y-%m-%d")
    formatted_current_datetime = f"{today} {timestamp}"
    current_datetime_obj = datetime.strptime(
        formatted_current_datetime, "%Y-%m-%d %H:%M:%S"
    )

    current_yields = {
        current_datetime_obj: [
            float(str(x["last"]).strip("%")) / 100 for x in current_yields_dict.values()
        ]
    }
    
    current_date = pd.Timestamp.now().normalize()
    months = [current_date - BMonthBegin(i) for i in range(2, num_of_months or 13)]
    monthly_historic_yields = [find_historical_yields(m) for m in months] if df_historical_par_yields is not None else []
    
    all_yields = [current_yields]
    
    if show_recent_spots_mins and df_recent_spots is not None:
        min_keys = df_recent_spots["Date"].values.tolist()
        for min_key in min_keys:
            yields = df_recent_spots.loc[
                df_recent_spots["Date"] == min_keys
            ]
            yields = yields.values.tolist()[0]
            del yields[:1]
            yields = [parse_to_float(y) / 100 for y in yields]
            all_yields.append({min_key: yields})

    if show_recents_days:
        yesterday, week, month, _ = get_business_days()
        yesterday_yields = find_historical_yields(yesterday)
        last_week_yields = find_historical_yields(week)
        last_month_yields = find_historical_yields(month)

        all_yields += [
            yesterday_yields,
            last_week_yields,
            last_month_yields,
        ] + monthly_historic_yields
    else:
        all_yields += monthly_historic_yields 

    cols = ["US1Y", "US2Y", "US3Y", "US5Y", "US7Y", "US10Y", "US20Y", "US30Y"]
    df_historical_monthly_yields = pd.DataFrame.from_dict(
        {list(d.keys())[0]: list(d.values())[0] for d in all_yields},
        orient="index",
        columns=cols,
    )
    df_historical_monthly_yields.index.name = "Date"
    df_historical_monthly_yields = df_historical_monthly_yields.reset_index()
    df_historical_monthly_yields_corr = pd.DataFrame(
        [
            (key, value)
            for item in all_yields
            for key, value in item.items()
        ],
        columns=["Date", "Values"],
    )
    df_historical_monthly_yields_corr.set_index("Date", inplace=True)

    def categorize_correlation(r):
        if r >= 0.7:
            return "Normal"
        elif 0.7 > r >= 0.35:
            return "Slight Normal"
        elif 0.35 > r > -0.35:
            return "Flat"
        elif -0.35 >= r > -0.7:
            return "Slight Inverted"
        elif r <= -0.7:
            return "Inverted"
        else:
            return "Unknown"

    # Pearson's correlation coefficient for each set of values
    correlations = []
    for values in df_historical_monthly_yields_corr["Values"]:
        if all(v is not None for v in values):
            index = np.arange(len(values))
            correlation = np.corrcoef(index, values)[0, 1]
        else:
            correlation = np.nan
        correlations.append(correlation)

    df_historical_monthly_yields_corr["Correlation"] = correlations
    df_historical_monthly_yields_corr["Category"] = df_historical_monthly_yields_corr[
        "Correlation"
    ].apply(categorize_correlation)
    df_historical_monthly_yields_corr[["Correlation", "Category"]]
    df_historical_monthly_yields = pd.merge(
        df_historical_monthly_yields, df_historical_monthly_yields_corr, on="Date"
    )
    df_historical_monthly_yields = df_historical_monthly_yields.drop("Values", axis=1)

    return df_historical_monthly_yields

In [4]:
def calc_daily_spreads(
    current_yields_dict: Dict[str, Dict[str, str]],
    mat1: str,
    mat2: str,
    print_out=False,
) -> Dict[str, int]:
    def s_to_f(s):
        return float(str(s).strip("%")) * 10

    spreads = {
        "spread_title": f"{mat1} - {mat2} (basis points)",
        "last_spread": round(
            (
                s_to_f(current_yields_dict[mat1]["last"])
                - s_to_f(current_yields_dict[mat2]["last"])
            )
            * 10,
            2,
        ),
        "open_spread": round(
            (
                s_to_f(current_yields_dict[mat1]["open"])
                - s_to_f(current_yields_dict[mat2]["open"])
            )
            * 10,
            2,
        ),
        "high_spread": round(
            (
                s_to_f(current_yields_dict[mat1]["high"])
                - s_to_f(current_yields_dict[mat2]["high"])
            )
            * 10,
            2,
        ),
        "low_spread": round(
            (
                s_to_f(current_yields_dict[mat1]["low"])
                - s_to_f(current_yields_dict[mat2]["low"])
            )
            * 10,
            2,
        ),
    }

    if print_out:
        print("timedate: ", current_yields_dict[mat1]["timedate"])
        print("name: ", f"{mat1} - {mat2} (basis points)")
        print("last: ", spreads["last_spread"])
        print("open: ", spreads["open_spread"])
        print("high: ", spreads["high_spread"])
        print("low: ", spreads["low_spread"])
        print(f"{mat1} del: ", current_yields_dict[mat1]["change"])
        print(f"{mat2} del: ", current_yields_dict[mat2]["change"])
        print("----------------")

    return spreads

In [5]:
def get_current_yield(mat, print_out=False) -> Dict[str, str | int]:
    res = requests.get(
        f"https://quote.cnbc.com/quote-html-webservice/restQuote/symbolType/symbol?symbols={mat}&requestMethod=itv&noform=1&partnerId=2&fund=1&exthrs=1&output=json&events=1"
    )
    if res.status_code != 200:
        return f"Bad Status: {res.status_code}"
    
    us_current_yield = res.json()["FormattedQuoteResult"]["FormattedQuote"][0]

    if print_out:
        print("timedate: ", us_current_yield["last_timedate"])
        print("name: ", us_current_yield["name"])
        print("last: ", us_current_yield["last"])
        print("open: ", us_current_yield["open"])
        print("high: ", us_current_yield["high"])
        print("low: ", us_current_yield["low"])
        print("change: ", us_current_yield["change"])
        print("----------------")

    return {
        "timedate": us_current_yield["last_timedate"],
        "name": us_current_yield["name"],
        "last": us_current_yield["last"],
        "open": us_current_yield["open"],
        "high": us_current_yield["high"],
        "low": us_current_yield["low"],
        "change": us_current_yield["change"],
    }

In [6]:
def yields_candlestick_plot(
    data: List[Dict[str, str]],
    use_candlestick=False,
    use_today=False,
    mat_label=None,
    just_close_plot=False,
):
    opens = [float(str((item["open"])).strip("%")) for item in data]
    highs = [float(str((item["high"])).strip("%")) for item in data]
    lows = [float(str((item["low"])).strip("%")) for item in data]
    closes = [float(str((item["close"])).strip("%")) for item in data]
    if use_today:
        trade_times = [
            datetime.fromtimestamp(int(item["tradeTimeinMills"]) / 1000).strftime(
                "%Y-%m-%d %H:%M:%S"
            )
            for item in data
            if datetime.fromtimestamp(int(item["tradeTimeinMills"]) / 1000)
            >= datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
        ]
    else:
        trade_times = [
            datetime.fromtimestamp(int(item["tradeTimeinMills"]) / 1000).strftime(
                "%Y-%m-%d %H:%M:%S"
            )
            for item in data
        ]

    if use_candlestick:
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=trade_times, open=opens, high=highs, low=lows, close=closes
                )
            ]
        )
    else:
        fig = go.Figure()
        if just_close_plot:
            fig.add_trace(
                go.Scatter(x=trade_times, y=closes, mode="lines", name="Close")
            )
        else:
            fig.add_trace(go.Scatter(x=trade_times, y=opens, mode="lines", name="Open"))
            fig.add_trace(go.Scatter(x=trade_times, y=highs, mode="lines", name="High"))
            fig.add_trace(go.Scatter(x=trade_times, y=lows, mode="lines", name="Low"))
            fig.add_trace(
                go.Scatter(x=trade_times, y=closes, mode="lines", name="Close")
            )

    fig.update_layout(
        title="Treasury Yield Data"
        if not mat_label
        else f"{mat_label} as of {datetime.now()}",
        xaxis_title="Trade Time",
        yaxis_title="Yield",
        xaxis_rangeslider_visible=False,
    )

    fig.show()

In [7]:
def periodic_work(interval: int, function: Callable):
    while True:
        function()
        time.sleep(interval)
        clear_output(wait=True)

In [76]:
def color_red(val):
    if isinstance(val, (int, float, complex)) and not isinstance(val, bool):
        color = "red" if val > 0 else "green"
    else:
        color = "none"

    return f"background-color: {color}"


def wrapper():
    current_yields_dict = {
        mat: get_current_yield(mat)
        for mat in ["US1Y", "US2Y", "US3Y", "US5Y", "US7Y", "US10Y", "US20Y", "US30Y"]
    }
    df = pd.DataFrame(current_yields_dict)
    df.drop(df.index[[1]], inplace=True)
    df.iloc[5] = pd.to_numeric(df.iloc[5], errors="coerce")
    df.iloc[5] = df.iloc[5] * 100

    styled_df = df.style.applymap(color_red)
    display(styled_df)
    return styled_df


# periodic_work(5, wrapper)

In [153]:
# ! python C:/Users/chris/trade/curr_pos/common/treasuries.py

df_historical_par_yields_2023 = pd.read_excel(r'C:\Users\chris\trade\curr_pos\treasuries\2023_daily_treasury_rates.xlsx', parse_dates=['Date'])
df_historical_par_yields_2022 = pd.read_excel(r'C:\Users\chris\trade\curr_pos\treasuries\2022_daily_treasury_rates.xlsx', parse_dates=['Date'])
df_historical_par_yields_2021 = pd.read_excel(r'C:\Users\chris\trade\curr_pos\treasuries\2021_daily_treasury_rates.xlsx', parse_dates=['Date'])
df_historical_par_yields_2020 = pd.read_excel(r'C:\Users\chris\trade\curr_pos\treasuries\2020_daily_treasury_rates.xlsx', parse_dates=['Date'])
df_historical_par_yields_2019 = pd.read_excel(r'C:\Users\chris\trade\curr_pos\treasuries\2019_daily_treasury_rates.xlsx', parse_dates=['Date'])
df_historical_par_yields = pd.concat([df_historical_par_yields_2023, df_historical_par_yields_2022, df_historical_par_yields_2022, df_historical_par_yields_2021, df_historical_par_yields_2020, df_historical_par_yields_2019])

df_historical_par_yields

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2023-12-07,5.53,5.54,5.44,5.47,5.36,5.05,4.58,4.31,4.11,4.16,4.14,4.42,4.25
1,2023-12-06,5.54,5.50,5.45,5.47,5.38,5.07,4.60,4.33,4.12,4.16,4.12,4.40,4.22
2,2023-12-05,5.54,5.51,5.45,5.46,5.37,5.06,4.57,4.33,4.14,4.20,4.18,4.48,4.30
3,2023-12-04,5.55,5.53,5.46,5.47,5.41,5.10,4.64,4.40,4.23,4.30,4.28,4.61,4.43
4,2023-12-01,5.55,5.53,5.43,5.45,5.33,5.05,4.56,4.31,4.14,4.22,4.22,4.58,4.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2019-01-08,2.40,2.42,2.46,NaN,2.54,2.60,2.58,2.57,2.58,2.63,2.73,2.88,3.00
246,2019-01-07,2.42,2.42,2.45,NaN,2.54,2.58,2.53,2.51,2.53,2.60,2.70,2.86,2.99
247,2019-01-04,2.40,2.42,2.42,NaN,2.51,2.57,2.50,2.47,2.49,2.56,2.67,2.83,2.98
248,2019-01-03,2.42,2.42,2.41,NaN,2.47,2.50,2.39,2.35,2.37,2.44,2.56,2.75,2.92


In [245]:
mats = ["US1Y", "US2Y", "US3Y", "US5Y", "US7Y", "US10Y", "US20Y", "US30Y"]
current_yields_dict = {mat: get_current_yield(mat) for mat in mats}
df = pd.DataFrame(current_yields_dict)
df.drop(df.index[[1]], inplace=True)
df.iloc[5] = pd.to_numeric(df.iloc[5], errors='coerce')
df.iloc[5] = df.iloc[5] * 100
styled_df = df.style.applymap(color_red)
styled_df

,US1Y,US2Y,US3Y,US5Y,US7Y,US10Y,US20Y,US30Y
timedate,5:05 PM EST,5:05 PM EST,5:05 PM EST,5:05 PM EST,5:05 PM EST,5:05 PM EST,5:05 PM EST,5:05 PM EST
last,5.139%,4.723%,4.46%,4.243%,4.271%,4.229%,4.489%,4.308%
open,5.076%,4.593%,4.34%,4.133%,4.179%,4.153%,4.435%,4.26%
high,5.156%,4.74%,4.486%,4.287%,4.326%,4.278%,4.548%,4.36%
low,5.068%,4.593%,4.34%,4.133%,4.177%,4.147%,4.427%,4.251%
change,8.300000,14.300000,13.900000,13.100000,11.500000,10.000000,7.000000,6.200000


In [244]:
# args = " ".join(str(x) for x in [69, 420])
# %run C:/Users/chris/trade/curr_pos/common/cnbc.py $args
! python C:/Users/chris/trade/curr_pos/common/cnbc.py 0 1 5 10 15 20 25 30 45 60 75 90 105 120 135 150 165 180 195 210 225 240 255 270 285 300 315 330 360 420 480 540 600

xlsx = pd.ExcelFile(r"C:\Users\chris\trade\curr_pos\rates\recent_spots.xlsx")
recent_spots_df_dict: Dict[str, pd.DataFrame] = {sheet_name: pd.read_excel(xlsx, sheet_name) for sheet_name in xlsx.sheet_names}

mats = ["Date", "US1Y", "US2Y", "US3Y", "US5Y", "US7Y", "US10Y", "US20Y", "US30Y"]
df_recent_spots = pd.DataFrame(columns=mats)
for mat, df in recent_spots_df_dict.items():
    row = [mat] + df["close"].values.tolist()
    df_recent_spots = pd.concat([pd.DataFrame([row], columns=df_recent_spots.columns), df_recent_spots], ignore_index=True)
    
# df_recent_spots = df_recent_spots.sort_values(by='Date', key=lambda x: print(x))
df_recent_spots["Date"] = df_recent_spots["Date"].str.replace('m', '').astype(float)
df_recent_spots = df_recent_spots.sort_values(by="Date", ascending=True)
df_recent_spots["Date"] = df_recent_spots["Date"].apply(lambda x: f"{str(x)}m")

styled_recent_spot_df = df_recent_spots.style.background_gradient(
    cmap=LinearSegmentedColormap.from_list("gr", ["g", "w", "r"], N=128),
    subset=[col for col in df_recent_spots.columns if col not in ["Date"]],
)
styled_recent_spot_df

{'US1Y': {'maturity': 'US1Y', '0.0m': {'open': '5.076%', 'high': '5.156%', 'low': '5.068%', 'close': 5.13, 'volume': None, 'tradeTime': '20231208170500', 'tradeTimeinMills': '1702073100000', '__typename': 'priceBarType', 'parsedTradeTime': datetime.datetime(2023, 12, 8, 22, 5), 'mat': 'US1Y'}, '1.0m': {'open': '5.076%', 'high': '5.156%', 'low': '5.068%', 'close': 5.13, 'volume': None, 'tradeTime': '20231208170500', 'tradeTimeinMills': '1702073100000', '__typename': 'priceBarType', 'parsedTradeTime': datetime.datetime(2023, 12, 8, 22, 5), 'mat': 'US1Y'}, '5.0m': {'open': '5.1360', 'high': '5.1360', 'low': '5.1360', 'close': '5.1360', 'volume': None, 'tradeTime': '20231208170000', 'tradeTimeinMills': '1702072800000', '__typename': 'priceBarType', 'parsedTradeTime': datetime.datetime(2023, 12, 8, 22, 0), 'mat': 'US1Y'}, '10.0m': {'open': '5.1390', 'high': '5.1390', 'low': '5.1390', 'close': '5.1390', 'volume': None, 'tradeTime': '20231208165700', 'tradeTimeinMills': '1702072620000', '__ty

,Date,US1Y,US2Y,US3Y,US5Y,US7Y,US10Y,US20Y,US30Y
32,0.0m,5.130000,4.720000,4.400000,4.240000,4.270000,4.220000,4.480000,4.300000
31,1.0m,5.130000,4.720000,4.457000,4.240000,4.270000,4.228000,4.489000,4.300000
30,5.0m,5.136000,4.725000,4.463000,4.241000,4.273000,4.228000,4.490000,4.305000
29,10.0m,5.139000,4.725000,4.460000,4.241000,4.270000,4.228000,4.489000,4.307000
28,15.0m,5.139000,4.723000,4.460000,4.243000,4.270000,4.228000,4.487000,4.305000
27,20.0m,5.139000,4.723000,4.457000,4.243000,4.271000,4.229000,4.490000,4.308000
26,25.0m,5.139000,4.721000,4.454000,4.241000,4.270000,4.228000,4.488000,4.307000
25,30.0m,5.139000,4.721000,4.454000,4.244000,4.271000,4.231000,4.493000,4.311000
24,45.0m,5.142000,4.725000,4.463000,4.248000,4.275000,4.235000,4.495000,4.313000
23,60.0m,5.139000,4.721000,4.454000,4.243000,4.270000,4.229000,4.488000,4.306000


In [234]:
plot_current_yields(current_yields_dict)
plot_current_yields_and_historicals(
    current_yields_dict=current_yields_dict,
    df_historical_par_yields=df_historical_par_yields,
    df_recent_spots=df_recent_spots,
    plot_yesterday=True,
    plot_week=True,
    # plot_month=True,
    # plot_year=True,
    # plot_five_min=True,
    # plot_ten_min=True,
    # plot_thirty_min=True,
    # plot_sixty_min=True,
    plot_120_min=True,
)

In [235]:
category_colors = {
    "Inverted": "red",
    "Slight Inverted": "darkorange",
    "Flat": "goldenrod",
    "Slight Normal": "lightgreen",
    "Normal": "green",
    "Unknown": "grey"  
}

def apply_color(val):
    color = category_colors.get(val, "white")  
    return f'background-color: {color}'

df_yield_corr = yields_corr(current_yields_dict=current_yields_dict, df_historical_par_yields=df_historical_par_yields, show_recents_days=True, num_of_months=36)
styled_df_yield_corr = df_yield_corr.style.background_gradient(cmap=LinearSegmentedColormap.from_list('rg', ["r", "w", "g"], N=256), subset=[col for col in df_yield_corr.columns if col not in ['Category', 'Date']])
styled_df_yield_corr = styled_df_yield_corr.applymap(apply_color, subset=['Category'])
styled_df_yield_corr

Date: 2023-01-02 00:00:00 probably not found:  list index out of range  trying next day


,Date,US1Y,US2Y,US3Y,US5Y,US7Y,US10Y,US20Y,US30Y,Correlation,Category
0,2023-12-08 15:34:00,0.051390,0.047230,0.044600,0.042460,0.042750,0.042330,0.044930,0.043130,-0.709896,Inverted
1,2023-12-07 00:00:00,0.050500,0.045800,0.043100,0.041100,0.041600,0.041400,0.044200,0.042500,-0.636624,Slight Inverted
2,2023-12-01 00:00:00,0.050500,0.045600,0.043100,0.041400,0.042200,0.042200,0.045800,0.044000,-0.457901,Slight Inverted
3,2023-11-08 00:00:00,0.053400,0.049300,0.046500,0.045100,0.045400,0.044900,0.048200,0.046400,-0.599772,Slight Inverted
4,2023-11-01 00:00:00,0.053700,0.049500,0.047600,0.046700,0.047500,0.047700,0.051300,0.049600,-0.230201,Flat
5,2023-10-02 00:00:00,0.054900,0.051200,0.048800,0.047200,0.047300,0.046900,0.050000,0.048100,-0.637564,Slight Inverted
6,2023-09-01 00:00:00,0.053600,0.048700,0.045700,0.042900,0.042700,0.041800,0.044800,0.042900,-0.777275,Inverted
7,2023-08-01 00:00:00,0.053800,0.049200,0.045700,0.042400,0.041500,0.040500,0.043000,0.041100,-0.850607,Inverted
8,2023-07-03 00:00:00,0.054300,0.049400,0.045600,0.041900,0.040300,0.038600,0.040800,0.038700,-0.903746,Inverted
9,2023-06-01 00:00:00,0.051100,0.043300,0.039800,0.037000,0.036600,0.036100,0.039800,0.038400,-0.693661,Slight Inverted


In [236]:
def cnbc_us_treasury_yields_fetcher(mat: str) -> List[Dict[str, str | int]]:
    res = requests.get(
        f"https://webql-redesign.cnbcfm.com/graphql?operationName=getQuoteChartData&variables=%7B%22symbol%22%3A%22{mat}%22%2C%22timeRange%22%3A%221D%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2261b6376df0a948ce77f977c69531a4a8ed6788c5ebcdd5edd29dd878ce879c8d%22%7D%7D"
    )
    return res.json()["data"]["chartData"]["priceBars"]

In [237]:
mats = ["US1Y", "US2Y", "US3Y", "US5Y", "US7Y", "US10Y", "US20Y", "US30Y"]
daily_yields_dict = {mat: cnbc_us_treasury_yields_fetcher(mat) for mat in mats}
[yields_candlestick_plot(data, use_candlestick=True, use_today=True, mat_label=mat) for mat, data in daily_yields_dict.items()]

[None, None, None, None, None, None, None, None]

In [238]:
def plot_spreads(mat1: str, mat2: str, daily_yields_dict: Dict[str, List[Dict[str, str | int]]]):
    chainmap_mat1 = {item["tradeTimeinMills"]:item for item in daily_yields_dict[mat1]}
    chainmap_mat2 = {item["tradeTimeinMills"]:item for item in daily_yields_dict[mat2]}

    def calc_spread(dict1, dict2, key):
        def format_float(item):
            return float(str((item)).strip("%")) 
        
        if key in dict1 and key in dict2:
            return {
                'open': format_float(dict1[key]['open']) - format_float(dict2[key]['open']),
                'high': format_float(dict1[key]['high']) - format_float(dict2[key]['high']),
                'low': format_float(dict1[key]['low']) - format_float(dict2[key]['low']),
                'close': format_float(dict1[key]['close']) - format_float(dict2[key]['close']),
                'tradeTimeinMills': key,
            }
        
        return None

    spread_dict = calc_daily_spreads(current_yields_dict, mat2, mat1)
    spread_df = pd.DataFrame([spread_dict])
    spread_df = spread_df.set_index('spread_title')
    display(spread_df.T)
        
    twos_tens_chainmap = [calc_spread(chainmap_mat2, chainmap_mat1, key) for key in chainmap_mat1.keys()]
    twos_tens_chainmap = list(filter(lambda item: item is not None, twos_tens_chainmap))
    yields_candlestick_plot(twos_tens_chainmap, use_candlestick=False, use_today=False, mat_label=f"{mat1}s{mat2}s", just_close_plot=True)

In [239]:
plot_spreads("US2Y", "US10Y", daily_yields_dict)
plot_spreads("US5Y", "US30Y", daily_yields_dict)

spread_title,US10Y - US2Y (basis points)
last_spread,-49.0
open_spread,-44.0
high_spread,-46.2
low_spread,-44.6


spread_title,US30Y - US5Y (basis points)
last_spread,6.7
open_spread,12.7
high_spread,7.3
low_spread,11.8
